In [1]:
%%capture
%pip install pandas numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt pandas_ta

In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

Matplotlib is building the font cache; this may take a moment.


In [27]:
# Load and manipulate most recent S&P 500 companies from Wikipedia (not survivorship-bias free)

# Loads several dfs, of which we only want the first
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Some symbols contain dots, replace with dash to prevent issues w yfinance
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# Get the unique symbols (added robustness)
symbols_list = sp500['Symbol'].unique().tolist()

# Set end date to current date
end_date = pd.Timestamp.today().date()

# Alternatively use a fixed date
# end_date = '2024-01-01'

# Set start date to be 8 years prior to end date
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# Download data from yfinance using tickers and date range (Time consuming)
# Stack method creates multi-index which is more convenient (reduces number of columns)
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

[*********************100%***********************]  503 of 503 completed


In [28]:
# Manipulate df, calculate features and technical indicators for each stock

# Assign new names for multi-index df
df.index.names = ['date', 'ticker']

# Assign column headers to strings (easier for manipulation later)
df.columns = df.columns.str.lower()

# Convert the `date` index to just the date part
df.index = df.index.set_levels([df.index.levels[0].date, df.index.levels[1]])

df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6


In [29]:
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-09-26 A        43.734436   46.570000   46.770000   46.160000   46.459999   
           AAPL     26.029970   28.219999   28.347500   27.887501   27.910000   
           ABBV     45.387062   64.070000   64.879997   64.019997   64.519997   
           ABT      36.004753   41.680000   42.020000   41.610001   42.009998   
           ACGL     26.486668   26.486668   26.653334   26.463333   26.653334   
...                       ...         ...         ...         ...         ...   
2024-09-20 XYL     134.809998  134.809998  136.309998  133.770004  135.820007   
           YUM     129.710007  129.710007  130.910004  128.720001  130.759995   
           ZBH     106.419998  106.419998  106.739998  105.010002  105.570000   
           ZBRA    366.760010  366.760010  367.559998  360.019989  362.429993   
           ZTS     192.639999  192.639999  194.550003  191.070007  192.940002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-09-26 A         2639100.0         -0.001326        NaN       NaN   
           AAPL    119477600.0         -0.001745        NaN       NaN   
           ABBV      5567700.0         -0.047706        NaN       NaN   
           ABT       6378200.0         -0.009144        NaN       NaN   
           ACGL      1023600.0          0.000010        NaN       NaN   
...                        ...               ...        ...       ...   
2024-09-20 XYL       2267000.0          0.000155  53.435794  4.847310   
           YUM       3272700.0          0.000117  40.091876  4.878913   
           ZBH       3920000.0          0.000109  43.013301  4.620909   
           ZBRA       596300.0          0.000160  65.622235  5.769405   
           ZTS       3520700.0          0.000162  60.241227  5.191393   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-09-26 A            NaN       NaN       NaN       NaN     115.419550  
           AAPL         NaN       NaN       NaN       NaN    3109.998364  
           ABBV         NaN       NaN       NaN       NaN     252.701546  
           ABT          NaN       NaN       NaN       NaN     229.645516  
           ACGL         NaN       NaN       NaN       NaN      27.111753  
...                     ...       ...       ...       ...            ...  
2024-09-20 XYL     4.894404  4.941498  1.038409 -0.081219     305.614264  
           YUM     4.902032  4.925151  0.403419 -0.750091     424.501939  
           ZBH     4.703192  4.785475 -0.617732 -0.543000     417.166393  
           ZBRA    5.837354  5.905304  0.276729  0.691584     218.698994  
           ZTS     5.243462  5.295530  0.648675  1.099930     678.227646  

[989692 rows x 14 columns]

In [ ]:
# new change